## Beispiel: URLs parallel abrufen

In [50]:
import threading
import requests

def fetch_url(url):
    print(f"Fetching {url}")
    response = requests.get(url)
    print(f"{url}: {len(response.content)} bytes")

urls = ['https://example.com', 'https://python.org', 'https://google.com']
threads = []

for url in urls:
    t = threading.Thread(target=fetch_url, args=(url,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

Fetching https://example.com
Fetching https://python.org
Fetching https://google.com
https://example.com: 1256 bytes
https://python.org: 49770 bytes
https://google.com: 17235 bytes


## Beispiel: CPU-intensive Berechnung

In [ ]:
import multiprocessing

def worker(n):
    print(f"Computing square of {n}")
    return n * n

if __name__ == '__main__':
    with multiprocessing.Pool(4) as pool:
        results = pool.map(worker, range(10))
    print(results)

Computing square of 3Computing square of 2Computing square of 1Computing square of 6Computing square of 0


Computing square of 5
Computing square of 4

Computing square of 7
Computing square of 9Computing square of 8


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


## Beispiel: Asynchrones Abrufen mehrerer URLs

In [51]:
import asyncio
import aiohttp

async def fetch(session, url):
    async with session.get(url) as response:
        print(f"{url}: {response.status}")
        return await response.text()

async def main():
    urls = ['https://example.com', 'https://python.org', 'https://google.com']
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        await asyncio.gather(*tasks)

# in Colab ist bereits ein Event Loop aktiv: await main()
# außerhalb Colab: asyncio.run(main())
await main()

https://example.com: 200
https://google.com: 200
https://python.org: 200


## Beispiel für eine Race Condition

In [53]:
import threading
import time

counter = 0

def increment():
    global counter
    temp = counter  # Read the value of counter
    time.sleep(0.0001)  # Delay to simulate a race condition
    counter = temp + 1  # Write back to counter

threads = []
for _ in range(1000):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(f'Counter: {counter}')

Counter: 423


## Lösung durch threading.Lock()

In [ ]:
import threading
import time

counter = 0
lock = threading.Lock()

def increment():
    global counter
    with lock:
        temp = counter
        time.sleep(0.0001)
        counter = temp + 1

threads = []
for _ in range(1000):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(f'Counter: {counter}')


Counter: 1000


## Beispiel für einen Deadlock

In [54]:
import threading
import time

lock_a = threading.Lock()
lock_b = threading.Lock()

def thread1():
    with lock_a:
        print("Thread 1: acquired lock A")
        time.sleep(0.1)
        print("Thread 1: waiting for lock B")
        with lock_b:
            print("Thread 1: acquired lock B")

def thread2():
    with lock_b:
        print("Thread 2: acquired lock B")
        time.sleep(0.1)
        print("Thread 2: waiting for lock A")
        with lock_a:
            print("Thread 2: acquired lock A")

t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()

Thread 1: acquired lock A
Thread 2: acquired lock B
Thread 1: waiting for lock B
Thread 2: waiting for lock A


KeyboardInterrupt: 